In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from huggingface_hub import login

login()

In [3]:
# DATASET CONFIG
DATASET='tyqiangz/multilingual-sentiments'
MODEL='lxyuan/distilbert-base-multilingual-cased-sentiments-student'
MIN_TOKENS=50
MAX_TOKENS=512
SAMPLES=10_000
MAX_VUS = 2000
TEXT_COLUMN='text'
DATASET_PATH='data/dataset.json'
K6_BIN = "/usr/bin/k6"



In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from src.process_dataset import tokenize_and_filter, sample_dataset, save_dataset

dataset = load_dataset(DATASET, 'all', split='train')

# Map numbers to text labels
label_mapping = {0: 'positive', 1: 'neutral', 2: 'negative'}
dataset = dataset.map(lambda example: {'label_text': label_mapping[example['label']]})


tokenizer = AutoTokenizer.from_pretrained(MODEL)

dataset = tokenize_and_filter(dataset, tokenizer, text_column='text', num_proc=8)
dataset = sample_dataset(dataset, n_samples=SAMPLES, min_tokens=MIN_TOKENS, max_tokens=MAX_TOKENS, seed=42)
save_dataset(dataset.select_columns(['text', 'label_text']), DATASET_PATH)

2025-01-25 09:04:38.336 | SUCCESS  | src.process_dataset:sample_dataset:29 - Sampled dataset down to 10000 samples
2025-01-25 09:04:38.459 | SUCCESS  | src.process_dataset:save_dataset:42 - Saved dataset to data/dataset.json


In [11]:
from dataclasses import dataclass, field, asdict
from typing import Dict

@dataclass
class InstanceConfig:
    repository: str
    accelerator: str
    instance_size: str
    instance_type: str
    custom_image: Dict[str, str]  # Dict of str -> str

@dataclass
class InfinityConfig:
    INFINITY_PORT: str = "80"
    INFINITY_ENGINE: str = "torch"
    INFINITY_BATCH_SIZE: str = "16"
    INFINITY_DTYPE: str = "auto"
    INFINITY_EMBEDDING_DTYPE: str = "float32"
    INFINITY_POOLING_METHOD: str = "auto"
    INFINITY_COMPILE: str = 'true'
    INFINITY_BETTERTRANSFORMER: str = 'true'
    INFINITY_MODEL_ID: str = "/repository"

    def to_dict(self) -> Dict[str, str]:
        """Converts the dataclass to a dictionary representation."""
        return asdict(self)

@dataclass
class ImageConfig:
    health_route: str = "/health"
    url: str = "michaelf34/infinity:0.0.75"
    env: InfinityConfig = field(default_factory=InfinityConfig)


In [12]:
def set_env(batch_size: int, image_config: ImageConfig) -> Dict[str, str]:
    config = asdict(image_config)
    config['env']["INFINITY_BATCH_SIZE"] = str(batch_size)
    return config

# Experiments

In [13]:
from src.deployment import deploy_endpoint
?deploy_endpoint

Signature: deploy_endpoint(instance_config)
Docstring:
Deploys or updates a Hugging Face inference endpoint.

This function checks if an existing inference endpoint with the specified name exists.
- If found, it updates the endpoint with the provided instance configuration.
- If not found, it creates a new inference endpoint with the given parameters.

Once the endpoint is updated or created, it waits until the endpoint is fully ready.

Args:
    instance_config (InstanceConfig): A dataclass containing instance configuration details,
                                      such as accelerator type, vendor, region, and instance size.

Returns:
    InferenceEndpoint: The deployed Hugging Face inference endpoint object.

Raises:
    Exception: If the endpoint creation process fails.
File:      /data/encoder-analysis/src/deployment.py
Type:      function

## GPUs

In [14]:
instance_config_experiment_dicts = [
    {
        'accelerator': 'gpu',
        'instance_size': 'x1',
        'instance_type': 'nvidia-t4'
    },
    {
        'accelerator': 'gpu',
        'instance_size': 'x1',
        'instance_type': 'nvidia-l4'
    },
]
instance_config_experiments = [InstanceConfig(repository=MODEL, **ic_exp_dict, custom_image=None) for ic_exp_dict in instance_config_experiment_dicts]

In [17]:
from src.k6 import call_k6, optimal_vus
from pathlib import Path
import copy
from time import sleep

template_file = "classification-analysis.js.j2"
output_file = Path("./generated").resolve()/"classification-analysis.js"

batch_sizes = [16, 32, 64, 128, 256, 512, 1024]
start_vus = 32

for og_instance_config_experiment in instance_config_experiments:
    for batch_size in batch_sizes:
        # Configure Infinity Settings, pass them up to the image
        infinty_config = InfinityConfig(INFINITY_BATCH_SIZE=str(batch_size))
        image_config = ImageConfig(env=infinty_config)

        # Add the image to the HW instance
        instance_config_experiment = copy.deepcopy(og_instance_config_experiment)
        instance_config_experiment.custom_image = image_config

        # Deploy the endpoint
        endpoint = deploy_endpoint(instance_config_experiment)

        args_dict = dict(
            endpoint=endpoint,
            total_requests=10,
            template_file=template_file,
            output_file=output_file,
            dataset_path=DATASET_PATH,
            k6_bin=K6_BIN
        )
        
        optimal_vus(max_vus=MAX_VUS, args_dict=args_dict, start_vus=start_vus)
        endpoint.delete()
        sleep(5)

2025-01-25 09:08:04.054 | INFO     | src.deployment:deploy_endpoint:84 - Creating inference endpoint...
2025-01-25 09:08:04.300 | INFO     | src.deployment:deploy_endpoint:105 - Waiting for endpoint to be ready...
2025-01-25 09:10:50.081 | SUCCESS  | src.deployment:deploy_endpoint:114 - Endpoint created successfully: hw=nvidia-t4	bs=16	Time taken: 2m 46.03s
2025-01-25 09:10:50.082 | INFO     | src.k6:optimal_vus:73 - Starting exponential search for optimal VUs
2025-01-25 09:10:50.082 | INFO     | src.k6:optimal_vus:75 - Testing with VUs: 32
2025-01-25 09:10:50.363 | INFO     | src.k6:call_k6:53 - Results written to /data/encoder-analysis/results/nvidia-t4/nvidia-t4_16_32.json
2025-01-25 09:10:50.366 | INFO     | src.k6:optimal_vus:78 - Throughput for 32 VUs: 0.00 req/sec
2025-01-25 09:10:50.367 | INFO     | src.k6:optimal_vus:75 - Testing with VUs: 64
2025-01-25 09:10:50.536 | INFO     | src.k6:call_k6:53 - Results written to /data/encoder-analysis/results/nvidia-t4/nvidia-t4_16_64.jso

KeyboardInterrupt: 

## CPU Experiments

In [23]:
instance_config_experiment_dicts = [
    {
        'accelerator': 'cpu',
        'instance_size': 'x1',
        'instance_type': 'intel-spr'
    }
]

instance_config_experiments = [InstanceConfig(repository=MODEL, **ic_exp_dict, custom_image=None) for ic_exp_dict in instance_config_experiment_dicts]

In [24]:
from loguru import logger
from itertools import product

batch_sizes = [1, 2, 4, 8, 16, 32, 64]
infinity_engine = ['torch', 'optimum']
start_vus = 1

cross_product = list(product(batch_sizes, infinity_engine))

for og_instance_config_experiment in instance_config_experiments:
    for batch_size, infinity_engine in cross_product:
        # Configure Infinity Settings, pass them up to the image
        infinty_config = InfinityConfig(INFINITY_BATCH_SIZE=str(batch_size))
        image_config = ImageConfig(env=infinty_config)

        # Add the image to the HW instance
        instance_config_experiment = copy.deepcopy(og_instance_config_experiment)
        instance_config_experiment.custom_image = image_config

        # Deploy the endpoint
        logger.info(f'Deploying with Batch Size:\t{batch_size}')
        logger.info(f'Deploying with Infinity Engine:\t{infinity_engine}')
        endpoint = deploy_endpoint(instance_config_experiment)

        args_dict = dict(
            endpoint=endpoint,
            total_requests=10_000,
            template_file=template_file,
            output_file=output_file,
            dataset_path=DATASET_PATH,
            k6_bin=K6_BIN
        )
        
        optimal_vus(max_vus=MAX_VUS, args_dict=args_dict, start_vus=start_vus)
        endpoint.delete()
        sleep(5)

2025-01-25 09:29:13.563 | INFO     | __main__:<module>:21 - Deploying with Batch Size:	1
2025-01-25 09:29:13.564 | INFO     | __main__:<module>:22 - Deploying with Infinity Engine:	torch
2025-01-25 09:29:13.564 | INFO     | src.deployment:deploy_endpoint:84 - Creating inference endpoint...
2025-01-25 09:29:13.663 | INFO     | src.deployment:deploy_endpoint:105 - Waiting for endpoint to be ready...
2025-01-25 09:30:08.950 | SUCCESS  | src.deployment:deploy_endpoint:114 - Endpoint created successfully: hw=intel-spr	bs=1	Time taken: 0m 55.39s
2025-01-25 09:30:08.951 | INFO     | src.k6:optimal_vus:73 - Starting exponential search for optimal VUs
2025-01-25 09:30:08.952 | INFO     | src.k6:optimal_vus:75 - Testing with VUs: 1
2025-01-25 09:31:10.609 | INFO     | src.k6:call_k6:53 - Results written to /data/encoder-analysis/results/intel-spr/intel-spr_1_1.json
2025-01-25 09:31:10.613 | INFO     | src.k6:optimal_vus:78 - Throughput for 1 VUs: 0.60 req/sec
2025-01-25 09:31:10.613 | INFO     |

KeyboardInterrupt: 